In [1]:
import torch
import numpy as np
import pandas as pd
import os
import datetime
from datetime import date, timedelta
from graph import Graph
from utils.data_utils import data_to_numpy

In [2]:
unlabelled = pd.read_csv('data/la_train_grid_with_weather.csv')
training_data = pd.read_csv('data/LA_DATA_2018_02_to_2018_06.csv')


In [3]:
def format_unlabeled_dataset(df):
    df = df.rename({'day' : 'DATE'}, axis = 1)

    cols = ['wind_x', 'wind_y', 'temperature', 'pressure', 'ceiling', 'dew',
                 'precipitation_duration' , 'mean_aod','min_aod','max_aod', 'visibility',                                               
                 'Latitudes','Longitudes','DATE']

    df = df[cols]
    return df
    

In [4]:
unlabelled = format_unlabeled_dataset(unlabelled)

In [5]:
# node_cols = 'visibility', 'precipitation_depth']
node_cols = ['wind_x', 'wind_y', 'temperature', 'pressure', 'ceiling', 'dew', 'precipitation_duration' , 'mean_aod','min_aod','max_aod', 'pm25']                                                 
edge_cols = ['wind_x', 'wind_y']
date_range = sorted(training_data['DATE'].unique(), key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

def assign_id(weather_data):
    ids = np.zeros(len(weather_data['Latitudes']))
    node_pos = list(weather_data.groupby(['Latitudes', 'Longitudes']).groups)
    for i, node in enumerate(node_pos):
        indices_lats = weather_data['Latitudes'].values == node_pos[i][0]
        indices_longs = weather_data['Longitudes'].values == node_pos[i][1]
        indices = indices_lats & indices_longs
        ids[np.argwhere(indices)] = int(i+1)
    weather_data['STATION'] = ids
    return weather_data

def get_unlabelled_subset(unlabeled_df, dates, node_cols, edge_cols, meta_cols):
    cols = list(set(node_cols + edge_cols + meta_cols))
    unlabeled_df = unlabeled_df[cols]
    unlabeled_df = unlabeled_df[unlabeled_df['DATE'].isin(dates)]
    unlabeled_df = assign_id(unlabeled_df)
    unlabeled_df['pm25'] = [np.nan for _ in range(len(unlabeled_df))]
    return unlabeled_df






# graph_node_features, graph_edge_features, graph_labels

In [6]:
node_cols = ['wind_x', 'wind_y', 'temperature', 'pressure', 'ceiling', 'dew', 'precipitation_duration' , 'mean_aod','min_aod','max_aod']                                                 
edge_cols = ['wind_x', 'wind_y']
meta_cols = ['DATE', 'Latitudes', 'Longitudes']
date_range = sorted(training_data['DATE'].unique(), key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

unlabeled_df = get_unlabelled_subset(unlabelled, date_range,  node_cols, edge_cols, meta_cols)

In [7]:
dataset = unlabeled_df.append(training_data, ignore_index=True)

In [9]:
graph_node_features, graph_edge_features, graph_labels = data_to_numpy(dataset, edge_cols, node_cols, pseudo_data = True)

Checkpt doesnt exist, making it
Creating checkpoint


In [10]:
graph_node_features.shape

(108, 565, 10)

In [8]:
dataset

,wind_x,min_aod,mean_aod,wind_y,dew,precipitation_duration,temperature,Latitudes,ceiling,Longitudes,max_aod,pressure,day,STATION,pm25,Unnamed: 0,DATE,visibility,precipitation_depth
0,NaN,0.117000,0.151435,NaN,NaN,NaN,NaN,-118.225777,NaN,33.664840,0.193000,10150.291670,2018-02-01,1.060000e+02,NaN,NaN,NaN,NaN,NaN
1,NaN,0.066000,0.097443,NaN,NaN,NaN,NaN,-118.225777,NaN,33.664840,0.147000,10185.125000,2018-02-03,1.060000e+02,NaN,NaN,NaN,NaN,NaN
2,NaN,0.068000,0.098783,NaN,NaN,NaN,NaN,-118.225777,NaN,33.664840,0.138000,10172.125000,2018-02-04,1.060000e+02,NaN,NaN,NaN,NaN,NaN
3,NaN,0.103000,0.140263,NaN,NaN,NaN,NaN,-118.225777,NaN,33.664840,0.435000,10154.958330,2018-02-05,1.060000e+02,NaN,NaN,NaN,NaN,NaN
4,NaN,0.144000,0.180686,NaN,NaN,NaN,NaN,-118.225777,NaN,33.664840,0.224000,10150.916670,2018-02-06,1.060000e+02,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50210,0.450819,0.199653,0.240690,0.541637,0.817843,0.04,0.823074,34.037858,1.000000,117.596957,0.410909,0.103103,NaN,7.470400e+10,16.815789,7337.0,2018-06-03,0.049762,0.0
50211,0.418716,0.104167,0.314764,0.389214,0.907599,0.04,0.789103,34.037858,0.766838,117.596957,0.449091,0.030051,NaN,7.470400e+10,18.605556,7338.0,2018-06-04,0.049244,0.0
50212,0.591639,0.281250,0.578224,0.489402,0.953616,0.04,0.627534,34.037858,0.542456,117.596957,0.801818,0.059614,NaN,7.470400e+10,17.115000,7339.0,2018-06-05,0.039891,0.0
50213,0.431580,0.097222,0.366079,0.504999,0.920696,0.04,0.615757,34.037858,0.479781,117.596957,0.798182,0.078427,NaN,7.470400e+10,19.071429,7340.0,2018-06-06,0.038846,0.0
